# BLG-407 Makine Öğrenmesi – Proje 3 (2025-26 Güz)

**Ad Soyad:**  **Melek ÇATAL**
**Okul Numarası:** **2212721039**
**GitHub Repo:**  **https://github.com/melekcatal/MLP_Flask.git**

## Proje Konusu
Bu projede, araç özelliklerine bağlı olarak araç satış fiyatını tahmin eden
çoklu doğrusal regresyon modeli geliştirilmiş ve model Flask tabanlı
bir web arayüzü ile sunulmuştur.


In [ ]:
# Google Drive, veri setine erişim sağlamak amacıyla Colab ortamına bağlanmış ve proje veri klasörü kontrol edilmiştir.

from google.colab import drive
drive.mount('/content/drive')

import os
os.listdir('/content/drive/MyDrive/MLP_Flask/data')


Mounted at /content/drive


['cars.csv']

In [ ]:
# Veri seti CSV formatında okunmuş, boyutu, ilk gözlemleri ve sütun isimleri incelenerek veri yapısı hakkında genel bir fikir edinilmiştir.

import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/MLP_Flask/data/cars.csv')
print("Shape:", df.shape)
display(df.head())
print(df.columns)


Shape: (8128, 13)


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque',
       'seats'],
      dtype='object')


## Feature Seçimi

Model karmaşıklığını azaltmak ve istatistiksel olarak daha anlamlı bir
regresyon modeli elde etmek amacıyla, toplam öznitelik sayısı maksimum
10 olacak şekilde sınırlandırılmıştır.

Bu kapsamda, araç fiyatını doğrudan etkilediği düşünülen yıl, kilometre,
motor özellikleri ve temel araç nitelikleri seçilmiştir. Çok fazla
kategori içeren veya temizlenmesi zor olan bazı değişkenler
(name, torque vb.) model dışında bırakılmıştır.


In [ ]:
# Hedef ve feature'ların belirlenmesi

features = [
    "year",
    "km_driven",
    "mileage",
    "engine",
    "max_power",
    "seats",
    "fuel",
    "transmission",
    "seller_type",
    "owner"
]

# Bağımsız değişkenler (X) ve hedef değişken (y) belirlenmiş, model eğitiminde kullanılacak veri yapısı oluşturulmuştur.

X = df[features]
y = df["selling_price"]

X.head()


,year,km_driven,mileage,engine,max_power,seats,fuel,transmission,seller_type,owner
0,2014,145500,23.4 kmpl,1248 CC,74 bhp,5.0,Diesel,Manual,Individual,First Owner
1,2014,120000,21.14 kmpl,1498 CC,103.52 bhp,5.0,Diesel,Manual,Individual,Second Owner
2,2006,140000,17.7 kmpl,1497 CC,78 bhp,5.0,Petrol,Manual,Individual,Third Owner
3,2010,127000,23.0 kmpl,1396 CC,90 bhp,5.0,Diesel,Manual,Individual,First Owner
4,2007,120000,16.1 kmpl,1298 CC,88.2 bhp,5.0,Petrol,Manual,Individual,First Owner


## Veri Ön İşleme – Sayısal Dönüşümler

Veri setinde bazı sayısal değişkenler birim bilgileriyle birlikte
string formatında tutulmaktadır. Bu değişkenlerin regresyon modeli
tarafından kullanılabilmesi için birim ifadeleri kaldırılarak
sayısal (numeric) forma dönüştürülmesi gerekmektedir.


In [ ]:
# Birim bilgisi içeren string değişkenler temizlenerek sayısal forma dönüştürülmüş ve sayısal dönüşüm sırasında hatalı değerler NaN olarak işaretlenmiştir.

# mileage: "23.4 kmpl" -> 23.4
X["mileage"] = X["mileage"].str.replace(" kmpl", "", regex=False)
X["mileage"] = X["mileage"].str.replace(" km/kg", "", regex=False)
X["mileage"] = pd.to_numeric(X["mileage"], errors="coerce")

# engine: "1248 CC" -> 1248
X["engine"] = X["engine"].str.replace(" CC", "", regex=False)
X["engine"] = pd.to_numeric(X["engine"], errors="coerce")

# max_power: "74 bhp" -> 74
X["max_power"] = X["max_power"].str.replace(" bhp", "", regex=False)
X["max_power"] = pd.to_numeric(X["max_power"], errors="coerce")

X.head()


/tmp/ipython-input-1298560633.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["mileage"] = X["mileage"].str.replace(" kmpl", "", regex=False)
/tmp/ipython-input-1298560633.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["mileage"] = X["mileage"].str.replace(" km/kg", "", regex=False)
/tmp/ipython-input-1298560633.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,year,km_driven,mileage,engine,max_power,seats,fuel,transmission,seller_type,owner
0,2014,145500,23.40,1248.0,74.00,5.0,Diesel,Manual,Individual,First Owner
1,2014,120000,21.14,1498.0,103.52,5.0,Diesel,Manual,Individual,Second Owner
2,2006,140000,17.70,1497.0,78.00,5.0,Petrol,Manual,Individual,Third Owner
3,2010,127000,23.00,1396.0,90.00,5.0,Diesel,Manual,Individual,First Owner
4,2007,120000,16.10,1298.0,88.20,5.0,Petrol,Manual,Individual,First Owner


In [ ]:
# Sayısal dönüşümler sonrasında oluşan eksik değerlerin sütun bazında dağılımı incelenmiştir.

X.isnull().sum()


,0
year,0
km_driven,0
mileage,221
engine,221
max_power,216
seats,221
fuel,0
transmission,0
seller_type,0
owner,0


## Kayıp Verilerin Ele Alınması

Sayısal dönüşüm işlemleri sonrasında bazı sütunlarda eksik değerler
oluşmuştur. Bu eksik değerler, veri setinin genel dağılımını bozmayacak
ve aykırı değerlere karşı daha dayanıklı olan median yöntemi ile
doldurulmuştur. Satır silme yöntemi tercih edilmemiştir, çünkü bu durum
veri kaybına yol açarak model performansını olumsuz etkileyebilir.


In [ ]:
from sklearn.impute import SimpleImputer

num_cols = ["mileage", "engine", "max_power", "seats"]

# Eksik sayısal değerler, aykırı değerlere karşı daha dayanıklı olması nedeniyle median yöntemi ile doldurulmuştur.

imputer = SimpleImputer(strategy="median")
X[num_cols] = imputer.fit_transform(X[num_cols])

# Kontrol
X.isnull().sum()


/tmp/ipython-input-4152541159.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_cols] = imputer.fit_transform(X[num_cols])


,0
year,0
km_driven,0
mileage,0
engine,0
max_power,0
seats,0
fuel,0
transmission,0
seller_type,0
owner,0


## Kategorik Verilerin Kodlanması

Veri setinde yer alan yakıt türü, şanzıman tipi, satıcı türü ve sahiplik
durumu gibi kategorik değişkenler sayısal bir sıralama içermediğinden,
One-Hot Encoding yöntemi kullanılarak modele dahil edilmiştir.


In [ ]:
# Nominal yapıya sahip kategorik değişkenler, çoklu doğrusal regresyon modeline uygun olacak şekilde One-Hot Encoding yöntemi ile sayısallaştırılmıştır.

X_encoded = pd.get_dummies(
    X,
    columns=["fuel", "transmission", "seller_type", "owner"],
    drop_first=True
)

X_encoded.head()


,year,km_driven,mileage,engine,max_power,seats,fuel_Diesel,fuel_LPG,fuel_Petrol,transmission_Manual,seller_type_Individual,seller_type_Trustmark Dealer,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,145500,23.40,1248.0,74.00,5.0,True,False,False,True,True,False,False,False,False,False
1,2014,120000,21.14,1498.0,103.52,5.0,True,False,False,True,True,False,False,True,False,False
2,2006,140000,17.70,1497.0,78.00,5.0,False,False,True,True,True,False,False,False,False,True
3,2010,127000,23.00,1396.0,90.00,5.0,True,False,False,True,True,False,False,False,False,False
4,2007,120000,16.10,1298.0,88.20,5.0,False,False,True,True,True,False,False,False,False,False


In [ ]:
# Veri seti, modelin genelleme yeteneğini değerlendirmek amacıyla eğitim ve test kümelerine ayrılmıştır.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y,
    test_size=0.2,
    random_state=42
)

X_train.shape, X_test.shape


((6502, 16), (1626, 16))

## Geriye Doğru Eleme (Backward Elimination)

Backward Elimination yöntemi, istatistiksel olarak anlamlı olmayan
özniteliklerin modelden çıkarılması amacıyla kullanılan bir yöntemdir.
Bu yöntemde, tüm özniteliklerle kurulan modelde p-value değeri 0.05'ten
büyük olan değişkenler sırasıyla modelden çıkarılır ve model yeniden
kurulur. Bu işlem, tüm öznitelikler istatistiksel olarak anlamlı hale
gelene kadar devam eder.


In [ ]:
# statsmodels kütüphanesi sayısal veri tipleri gerektirdiğinden, tüm öznitelikler float tipine dönüştürülmüştür.

X_encoded = X_encoded.astype(float)


In [ ]:
import statsmodels.api as sm

# Sabit terim eklenerek OLS tabanlı çoklu doğrusal regresyon modeli kurulmuş ve istatistiksel özet sonuçları elde edilmiştir

X_train_sm = sm.add_constant(X_train.astype(float))
model_ols = sm.OLS(y_train, X_train_sm).fit()

model_ols.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          selling_price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.681
Method:                 Least Squares   F-statistic:                     867.0
Date:                Mon, 15 Dec 2025   Prob (F-statistic):               0.00
Time:                        10:44:45   Log-Likelihood:                -93928.
No. Observations:                6502   AIC:                         1.879e+05
Df Residuals:                    6485   BIC:                         1.880e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                        -6.403e+07   3.91e+06    -16.388      0.000   -7.17e+07   -5.64e+07
year                           3.19e+04   1953.060     16.333      0.000    2.81e+04    3.57e+04
km_driven                       -0.9866      0.114     -8.662      0.000      -1.210      -0.763
mileage                       8996.2989   2330.752      3.860      0.000    4427.255    1.36e+04
engine                          66.2263     26.055      2.542      0.011      15.150     117.303
max_power                     1.253e+04    283.372     44.220      0.000     1.2e+04    1.31e+04
seats                        -3.989e+04   8771.483     -4.547      0.000   -5.71e+04   -2.27e+04
fuel_Diesel                  -2.351e+04   6.81e+04     -0.345      0.730   -1.57e+05     1.1e+05
fuel_LPG                      1.393e+05   1.06e+05      1.309      0.191   -6.94e+04    3.48e+05
fuel_Petrol                  -1.008e+05   6.83e+04     -1.476      0.140   -2.35e+05    3.31e+04
transmission_Manual          -4.713e+05   2.18e+04    -21.640      0.000   -5.14e+05   -4.29e+05
seller_type_Individual       -2.494e+05   1.83e+04    -13.600      0.000   -2.85e+05   -2.13e+05
seller_type_Trustmark Dealer -3.469e+05   3.78e+04     -9.179      0.000   -4.21e+05   -2.73e+05
owner_Fourth & Above Owner    9387.7377   4.06e+04      0.231      0.817   -7.02e+04     8.9e+04
owner_Second Owner             -5.4e+04   1.46e+04     -3.705      0.000   -8.26e+04   -2.54e+04
owner_Test Drive Car          1.686e+06   2.28e+05      7.383      0.000    1.24e+06    2.13e+06
owner_Third Owner            -3.857e+04   2.49e+04     -1.547      0.122   -8.75e+04    1.03e+04
==============================================================================
Omnibus:                     3576.052   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            62054.973
Skew:                           2.254   Prob(JB):                         0.00
Kurtosis:                      17.448   Cond. No.                     6.31e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.31e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Backward Elimination – 1. Adım

Kurulan ilk modelde p-value değeri 0.05'ten büyük olan değişkenler
incelenmiştir. En yüksek p-value değerine sahip olan
**owner_Fourth & Above Owner** değişkeni istatistiksel olarak anlamlı
bulunmadığı için modelden çıkarılmıştır.


In [ ]:
# Modelde p-value değeri 0.05’ten büyük olan ilgili değişken, istatistiksel olarak anlamlı bulunmadığı için modelden çıkarılmıştır.

# 1. eleme: owner_Fourth & Above Owner

X_train_be = X_train.drop(columns=["owner_Fourth & Above Owner"])

# statsmodels için float zorunlu
X_train_be = X_train_be.astype(float)

# sabit terim ekle
X_train_sm = sm.add_constant(X_train_be)

# OLS
model_ols = sm.OLS(y_train, X_train_sm).fit()

model_ols.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          selling_price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.681
Method:                 Least Squares   F-statistic:                     924.9
Date:                Mon, 15 Dec 2025   Prob (F-statistic):               0.00
Time:                        10:47:44   Log-Likelihood:                -93928.
No. Observations:                6502   AIC:                         1.879e+05
Df Residuals:                    6486   BIC:                         1.880e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                        -6.383e+07   3.81e+06    -16.742      0.000   -7.13e+07   -5.64e+07
year                           3.18e+04   1906.882     16.677      0.000    2.81e+04    3.55e+04
km_driven                       -0.9858      0.114     -8.659      0.000      -1.209      -0.763
mileage                       8992.1917   2330.515      3.858      0.000    4423.614    1.36e+04
engine                          66.1266     26.050      2.538      0.011      15.061     117.193
max_power                     1.253e+04    283.331     44.229      0.000     1.2e+04    1.31e+04
seats                        -3.986e+04   8770.200     -4.545      0.000   -5.71e+04   -2.27e+04
fuel_Diesel                  -2.377e+04   6.81e+04     -0.349      0.727   -1.57e+05     1.1e+05
fuel_LPG                      1.388e+05   1.06e+05      1.304      0.192   -6.98e+04    3.47e+05
fuel_Petrol                  -1.011e+05   6.83e+04     -1.480      0.139   -2.35e+05    3.28e+04
transmission_Manual          -4.714e+05   2.18e+04    -21.646      0.000   -5.14e+05   -4.29e+05
seller_type_Individual       -2.492e+05   1.83e+04    -13.605      0.000   -2.85e+05   -2.13e+05
seller_type_Trustmark Dealer -3.468e+05   3.78e+04     -9.178      0.000   -4.21e+05   -2.73e+05
owner_Second Owner           -5.468e+04   1.43e+04     -3.831      0.000   -8.27e+04   -2.67e+04
owner_Test Drive Car          1.686e+06   2.28e+05      7.385      0.000    1.24e+06    2.13e+06
owner_Third Owner            -3.948e+04   2.46e+04     -1.603      0.109   -8.78e+04    8806.189
==============================================================================
Omnibus:                     3576.353   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            62073.351
Skew:                           2.254   Prob(JB):                         0.00
Kurtosis:                      17.450   Cond. No.                     6.16e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.16e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Backward Elimination – 2. Adım

Birinci eleme sonrasında yeniden kurulan modelde, p-value değeri en yüksek
olan **fuel_Diesel** değişkeni istatistiksel olarak anlamlı
bulunmadığı için modelden çıkarılmıştır.


In [ ]:
# Modelde p-value değeri 0.05’ten büyük olan ilgili değişken, istatistiksel olarak anlamlı bulunmadığı için modelden çıkarılmıştır.

# 2. eleme: fuel_Diesel

X_train_be = X_train_be.drop(columns=["fuel_Diesel"])

# statsmodels için float zorunlu
X_train_be = X_train_be.astype(float)

# sabit terim ekle
X_train_sm = sm.add_constant(X_train_be)

# OLS
model_ols = sm.OLS(y_train, X_train_sm).fit()

model_ols.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          selling_price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.681
Method:                 Least Squares   F-statistic:                     991.1
Date:                Mon, 15 Dec 2025   Prob (F-statistic):               0.00
Time:                        10:48:40   Log-Likelihood:                -93928.
No. Observations:                6502   AIC:                         1.879e+05
Df Residuals:                    6487   BIC:                         1.880e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                        -6.388e+07   3.81e+06    -16.767      0.000   -7.13e+07   -5.64e+07
year                          3.181e+04   1906.373     16.688      0.000    2.81e+04    3.56e+04
km_driven                       -0.9864      0.114     -8.666      0.000      -1.210      -0.763
mileage                       8983.9820   2330.238      3.855      0.000    4415.947    1.36e+04
engine                          65.7749     26.028      2.527      0.012      14.751     116.799
max_power                     1.253e+04    283.102     44.251      0.000     1.2e+04    1.31e+04
seats                        -3.992e+04   8768.273     -4.552      0.000   -5.71e+04   -2.27e+04
fuel_LPG                      1.619e+05   8.33e+04      1.945      0.052   -1315.547    3.25e+05
fuel_Petrol                  -7.796e+04   1.63e+04     -4.779      0.000    -1.1e+05    -4.6e+04
transmission_Manual          -4.716e+05   2.18e+04    -21.660      0.000   -5.14e+05   -4.29e+05
seller_type_Individual        -2.49e+05   1.83e+04    -13.601      0.000   -2.85e+05   -2.13e+05
seller_type_Trustmark Dealer -3.466e+05   3.78e+04     -9.174      0.000   -4.21e+05   -2.73e+05
owner_Second Owner           -5.471e+04   1.43e+04     -3.833      0.000   -8.27e+04   -2.67e+04
owner_Test Drive Car          1.686e+06   2.28e+05      7.387      0.000    1.24e+06    2.13e+06
owner_Third Owner             -3.95e+04   2.46e+04     -1.604      0.109   -8.78e+04    8779.035
==============================================================================
Omnibus:                     3577.419   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            62110.232
Skew:                           2.255   Prob(JB):                         0.00
Kurtosis:                      17.454   Cond. No.                     6.16e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.16e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Backward Elimination – 3. Adım

İkinci eleme sonrasında kurulan modelde, p-value değeri 0.05 sınırının
üzerinde olan **owner_Third Owner** değişkeni istatistiksel olarak
anlamlı bulunmadığı için modelden çıkarılmıştır.


In [ ]:
# Modelde p-value değeri 0.05’ten büyük olan ilgili değişken, istatistiksel olarak anlamlı bulunmadığı için modelden çıkarılmıştır.

# 3. eleme: owner_Third Owner

X_train_be = X_train_be.drop(columns=["owner_Third Owner"])

# statsmodels için float zorunlu
X_train_be = X_train_be.astype(float)

# sabit terim ekle
X_train_sm = sm.add_constant(X_train_be)

# OLS
model_ols = sm.OLS(y_train, X_train_sm).fit()

model_ols.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          selling_price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.681
Method:                 Least Squares   F-statistic:                     1067.
Date:                Mon, 15 Dec 2025   Prob (F-statistic):               0.00
Time:                        10:49:57   Log-Likelihood:                -93929.
No. Observations:                6502   AIC:                         1.879e+05
Df Residuals:                    6488   BIC:                         1.880e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                        -6.549e+07   3.68e+06    -17.819      0.000   -7.27e+07   -5.83e+07
year                          3.261e+04   1840.439     17.720      0.000     2.9e+04    3.62e+04
km_driven                       -0.9952      0.114     -8.753      0.000      -1.218      -0.772
mileage                       9042.8753   2330.231      3.881      0.000    4474.854    1.36e+04
engine                          65.9121     26.031      2.532      0.011      14.882     116.942
max_power                     1.252e+04    283.098     44.225      0.000     1.2e+04    1.31e+04
seats                        -3.959e+04   8767.050     -4.516      0.000   -5.68e+04   -2.24e+04
fuel_LPG                      1.628e+05   8.33e+04      1.955      0.051    -455.282    3.26e+05
fuel_Petrol                  -7.783e+04   1.63e+04     -4.770      0.000    -1.1e+05   -4.58e+04
transmission_Manual          -4.715e+05   2.18e+04    -21.656      0.000   -5.14e+05   -4.29e+05
seller_type_Individual       -2.513e+05   1.83e+04    -13.763      0.000   -2.87e+05   -2.15e+05
seller_type_Trustmark Dealer -3.472e+05   3.78e+04     -9.188      0.000   -4.21e+05   -2.73e+05
owner_Second Owner           -4.831e+04   1.37e+04     -3.525      0.000   -7.52e+04   -2.14e+04
owner_Test Drive Car          1.685e+06   2.28e+05      7.379      0.000    1.24e+06    2.13e+06
==============================================================================
Omnibus:                     3574.791   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            61965.653
Skew:                           2.253   Prob(JB):                         0.00
Kurtosis:                      17.437   Cond. No.                     5.94e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.94e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Backward Elimination – 4. Adım (Final)

Üçüncü eleme sonrasında kurulan modelde, p-value değeri 0.05 sınırına
çok yakın olmakla birlikte bu sınırın üzerinde kalan **fuel_LPG**
değişkeni, istatistiksel anlamlılık kriterine uymadığı için modelden
çıkarılmıştır. Bu eleme ile birlikte modelde yer alan tüm değişkenler
istatistiksel olarak anlamlı hale gelmiştir.


In [ ]:
# Modelde p-value değeri 0.05’ten büyük olan ilgili değişken, istatistiksel olarak anlamlı bulunmadığı için modelden çıkarılmıştır.

# 4. ve final eleme: fuel_LPG

X_train_be = X_train_be.drop(columns=["fuel_LPG"])

# statsmodels için float zorunlu
X_train_be = X_train_be.astype(float)

# sabit terim ekle
X_train_sm = sm.add_constant(X_train_be)

# Final OLS modeli
model_ols = sm.OLS(y_train, X_train_sm).fit()

model_ols.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          selling_price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.681
Method:                 Least Squares   F-statistic:                     1155.
Date:                Mon, 15 Dec 2025   Prob (F-statistic):               0.00
Time:                        10:51:09   Log-Likelihood:                -93931.
No. Observations:                6502   AIC:                         1.879e+05
Df Residuals:                    6489   BIC:                         1.880e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                        -6.521e+07   3.67e+06    -17.752      0.000   -7.24e+07    -5.8e+07
year                          3.248e+04   1839.646     17.657      0.000    2.89e+04    3.61e+04
km_driven                       -0.9999      0.114     -8.795      0.000      -1.223      -0.777
mileage                       8593.2773   2319.358      3.705      0.000    4046.572    1.31e+04
engine                          61.0305     25.917      2.355      0.019      10.225     111.836
max_power                     1.251e+04    283.121     44.190      0.000     1.2e+04    1.31e+04
seats                        -3.989e+04   8767.664     -4.549      0.000   -5.71e+04   -2.27e+04
fuel_Petrol                  -8.253e+04   1.61e+04     -5.113      0.000   -1.14e+05   -5.09e+04
transmission_Manual          -4.725e+05   2.18e+04    -21.699      0.000   -5.15e+05    -4.3e+05
seller_type_Individual       -2.514e+05   1.83e+04    -13.765      0.000   -2.87e+05   -2.16e+05
seller_type_Trustmark Dealer -3.459e+05   3.78e+04     -9.153      0.000    -4.2e+05   -2.72e+05
owner_Second Owner           -4.855e+04   1.37e+04     -3.542      0.000   -7.54e+04   -2.17e+04
owner_Test Drive Car          1.686e+06   2.28e+05      7.383      0.000    1.24e+06    2.13e+06
==============================================================================
Omnibus:                     3578.210   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            62119.819
Skew:                           2.255   Prob(JB):                         0.00
Kurtosis:                      17.455   Cond. No.                     5.93e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.93e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Backward Elimination Sonuçları

Backward Elimination yöntemi sonucunda, istatistiksel olarak anlamlı
olmayan öznitelikler sırasıyla modelden çıkarılmıştır. Final modelde
yer alan tüm değişkenlerin p-value değerleri 0.05’in altında olup,
model istatistiksel olarak anlamlı bir yapıya sahiptir.


In [ ]:
final_features_10 = [
    "year",
    "km_driven",
    "mileage",
    "engine",
    "max_power",
    "seats",
    "fuel_Petrol",
    "transmission_Manual",
    "seller_type_Individual",
    "owner_Second Owner"
]

X_train_10 = X_train[final_features_10].astype(float)
X_test_10  = X_test[final_features_10].astype(float)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Backward Elimination sonucunda elde edilen 10 öznitelik ile final çoklu doğrusal regresyon modeli eğitilmiştir.

lr_model_10 = LinearRegression()
lr_model_10.fit(X_train_10, y_train)


LinearRegression()

In [ ]:
# Model performansı, test veri seti üzerinde R², MAE ve MSE metrikleri kullanılarak değerlendirilmiştir.

y_pred_10 = lr_model_10.predict(X_test_10)

r2_10  = r2_score(y_test, y_pred_10)
mae_10 = mean_absolute_error(y_test, y_pred_10)
mse_10 = mean_squared_error(y_test, y_pred_10)

r2_10, mae_10, mse_10


(0.6832867079745404, 269209.2763688771, 207600634992.02927)

## Model Değerlendirme (10 Öznitelik ile)

Ödev isterlerine uygun olarak final modelde maksimum 10 öznitelik
kullanılmıştır. Model, test veri seti üzerinde R², MAE ve MSE metrikleri
ile değerlendirilmiş ve benzer doğruluk seviyesinin korunduğu
gözlemlenmiştir.


In [ ]:
import pickle

# Eğitilen model ve öznitelik sırası, Flask uygulamasında kullanılmak üzere pickle formatında kaydedilmiştir.

# Model
with open("model.pkl", "wb") as f:
    pickle.dump(lr_model_10, f)

# Feature sırası (Flask'ta birebir kullanılacak)
with open("features.pkl", "wb") as f:
    pickle.dump(final_features_10, f)


## Model Değerlendirme Sonuçları

Final model, test veri seti üzerinde R², MAE ve MSE metrikleri ile
değerlendirilmiştir. Elde edilen R² değeri, modelin araç fiyatlarındaki
varyansın yaklaşık %68’ini açıkladığını göstermektedir. MAE ve MSE
değerleri, modelin tahmin hatalarının kabul edilebilir seviyede
olduğunu ortaya koymaktadır.

Bu sonuçlar, modelin genelleme yeteneğinin yeterli olduğunu ve çoklu
doğrusal regresyon yaklaşımının bu problem için uygun olduğunu
göstermektedir.


In [ ]:
import pickle

# Eğitilmiş model
with open("model.pkl", "wb") as f:
    pickle.dump(lr_model_10, f)

# Feature sırası (Flask için kritik)
with open("features.pkl", "wb") as f:
    pickle.dump(final_features_10, f)


In [ ]:
import os
os.getcwd()
os.listdir()


['.config', 'model.pkl', 'drive', 'features.pkl', 'sample_data']

In [ ]:
import os

os.makedirs('/content/drive/MyDrive/MLP_Flask/templates', exist_ok=True)


In [ ]:
import shutil

shutil.move('/content/model.pkl',
            '/content/drive/MyDrive/MLP_Flask/model.pkl')

shutil.move('/content/features.pkl',
            '/content/drive/MyDrive/MLP_Flask/features.pkl')


'/content/drive/MyDrive/MLP_Flask/features.pkl'

In [ ]:
os.listdir('/content/drive/MyDrive/MLP_Flask')


['data', 'templates', 'model.pkl', 'features.pkl']